# Figures for Chapter on SAR

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import os
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == "Figures":
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.plotting_functions import plot_discrete_raster
write_fig = True

## Landuse and SAR 2D image

In [ ]:
s1_xr = xr.open_dataset('data/g0_OpenEO/g0_zwalm_landuse.nc', decode_coords= 'all') #change to gamma0
s1_xr = s1_xr.chunk({'t':20})
s1_xr['VV_db'] = 10 * np.log10(s1_xr['VV'])
s1_xr

In [ ]:
colormap = colors.ListedColormap(
            [
                "#000000",
                "#387b00",
                "#64cf1b",
                 "#a47158",
                "#00bfff",
            ]
        )
labels = [
    'Urban',
    'Forest',
    'Pasture',
    'Agriculture',
    'Water'
]

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (11.5,5),constrained_layout = True)
s1_xr['VV_db'].isel(t=0).plot(ax = axes[0],  cbar_kwargs = {'label':r'$\gamma^0$ [dB]'}, rasterized = True)
axes[0].set_aspect('equal','box')
axes[0].set_title('(a)')
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
# s1_xr['landuse'].plot(ax = axes[1])
# axes[1].set_aspect('equal','box')
fig, ax_1 = plot_discrete_raster(
    fig, axes[1], s1_xr['landuse'].values, bounds = list(s1_xr.rio.bounds()), 
    labels = labels, cmap = colormap, rasterized = True
)
ax_1.set_aspect('equal','box')
ax_1.set_xlabel('x [m]')
ax_1.set_ylabel('y [m]')
ax_1.set_title('(b)')
figures_SAR_path = Path("Figures/Figures_chapter_SAR")
if not os.path.exists(figures_SAR_path):
    os.makedirs(figures_SAR_path)
if write_fig:
    fig.savefig(figures_SAR_path/'gamma0_landuse_example.pdf',format = 'pdf')

Plot of gamma0 data seperately

In [ ]:
font_size = 13
plt.rcParams.update({'font.size': font_size})
fig, ax = plt.subplots(constrained_layout = True)
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.tick_params(axis='x', labelrotation=25)
s1_xr['VV_db'].isel(t=0).plot(ax = ax,  cbar_kwargs = {'label':r'$\gamma^0$ [dB]'}, rasterized = True)
ax.set_title('')
pad = Path('Figures/presentation_12_04')
if not os.path.exists(pad):
    os.makedirs(pad)
if write_fig:
    fig.savefig(pad/'gamma0_example.svg', format = 'svg')
plt.rcParams.update(matplotlib.rcParamsDefault)

In [ ]:
print(fig.get_figheight()); print(fig.get_figwidth())

## Timeseries

Read in the data

In [ ]:
s1_pd_g0 = pd.read_csv('data/g0_OpenEO/s1_g0_timeseries.csv',index_col ='t', parse_dates=True)
s1_pd = pd.read_csv('data/s0_OpenEO/s1_timeseries.csv',index_col ='t', parse_dates=True)

Visualisations

In [ ]:
s1_pd.head()

In [ ]:
s1_pd_ascending = s1_pd[s1_pd['Orbitdirection'] == 'ascending']
s1_pd_descending = s1_pd[s1_pd['Orbitdirection'] == 'descending']
cols = s1_pd.columns
VV_cols = cols[cols.str.startswith('VV')]
VH_cols = cols[cols.str.startswith('VH')]
names_legend = [name[2:] for name in VV_cols]
print(names_legend)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (10.5,7),constrained_layout = True)

#ascending
s1_pd_ascending[VV_cols].plot.line(ax = ax1)
ax1.set_title('Ascending')
ax1.set_ylabel(r'$\sigma^0_{VV}$ [dB]')
ax1.legend(names_legend, loc = 'upper right')
ax1.set_xlabel('Timestamp')

#descending
s1_pd_descending[VV_cols].plot.line(ax = ax2)
ax2.set_ylabel(r'$\sigma^0_{VV}$ [dB]')
ax2.set_title('Descending')
ax2.legend(names_legend, loc = 'upper right')
ax2.set_xlabel('Timestamp')
if write_fig:
    fig.savefig(figures_SAR_path/'sigma0_timeseries.pdf',format = 'pdf')

In [ ]:
fig

For presentation

In [ ]:
s1_pd_g0_ascending = s1_pd_g0[s1_pd_g0['Orbitdirection'] == 'ascending']
s1_pd_g0_descending = s1_pd_g0[s1_pd_g0['Orbitdirection'] == 'descending']
#as en example: take VV and ascending! 
s1_pd_g0_ascending_VV = s1_pd_g0_ascending.iloc[:,s1_pd_g0_ascending.columns.str.startswith('VV')]
s1_pd_g0_ascending_VV = s1_pd_g0_ascending_VV.iloc[:,~s1_pd_g0_ascending_VV.columns.str.endswith(('past_agr','avg'))]
s1_pd_g0_ascending_VV.head()

In [ ]:
font_size = 13
plt.rcParams.update({'font.size': font_size})
fig, ax = plt.subplots(figsize = (8,5),constrained_layout = True)
s1_pd_g0_ascending_VV.plot(cmap = colormap, ax = ax)
ax.legend(['Urbaan','Bos','Weiland','Landbouw','Water'])
ax.set_xlabel('Tijd')
ax.set_ylabel(r'Ascending $\gamma^0_{VV}$ [dB]')
display(fig)
if write_fig:
    fig.savefig(pad/'gamma0_VV_asc.svg', format = 'svg')
plt.rcParams.update(matplotlib.rcParamsDefault)

For dissertation: combination of 4 figures: gamma0_VV_asc, gamma0_VV_desc, gamma_0_VV_asc and gamma_0_VV_desc

In [ ]:
names_3_uses = ['Forest','Pasture','Agriculture']
colors_3_uses = colors.ListedColormap(["#387b00","#64cf1b","#a47158"])
VV_cols_3 = VV_cols[VV_cols.str.endswith(tuple(names_3_uses))]
VH_cols_3 = VH_cols[VH_cols.str.endswith(tuple(names_3_uses))]
fig, axes = plt.subplots(4,1,figsize = (10,10),constrained_layout = True)
#VV plots
s1_pd_g0_ascending[VV_cols_3].plot(ax= axes[0], cmap = colors_3_uses)
axes[0].set_xlabel('')
axes[0].set_ylabel(r'$\gamma^0_{\rm VV,asc}$ [dB]')
axes[0].legend(names_3_uses, loc='lower left')
s1_pd_g0_descending[VV_cols_3].plot(ax=axes[1], cmap = colors_3_uses)
axes[1].set_xlabel('')
axes[1].set_ylabel(r'$\gamma^0_{\rm VV,desc}$ [dB]')
axes[1].legend(names_3_uses, loc='lower left')

#VH plots
s1_pd_g0_ascending[VH_cols_3].plot(ax= axes[2], cmap = colors_3_uses)
axes[2].set_xlabel('')
axes[2].set_ylabel(r'$\gamma^0_{\rm VH,asc}$ [dB]')
axes[2].legend(names_3_uses, loc='lower left')
s1_pd_g0_descending[VH_cols_3].plot(ax=axes[3], cmap = colors_3_uses)
axes[3].set_xlabel('')
axes[3].set_ylabel(r'$\gamma^0_{\rm VH,desc}$ [dB]')
axes[3].legend(names_3_uses, loc='lower left')
fig.supxlabel('Timestamp')
if write_fig:
    fig.savefig('Figures/Figures_chapter_SAR/g0_timeseries_vierluik.pdf', format = 'pdf')
fig

Repeat the above figure for the $\sigma^0$ data: used in Appendix

In [ ]:
fig, axes = plt.subplots(4,1,figsize = (10,10),constrained_layout = True)
#VV plots
s1_pd_ascending[VV_cols_3].plot(ax= axes[0], cmap = colors_3_uses)
axes[0].set_xlabel('')
axes[0].set_ylabel(r'$\sigma^0_{\rm VV,asc}$ [dB]')
axes[0].legend(names_3_uses, loc='lower left')
s1_pd_descending[VV_cols_3].plot(ax=axes[1], cmap = colors_3_uses)
axes[1].set_xlabel('')
axes[1].set_ylabel(r'$\sigma^0_{\rm VV,desc}$ [dB]')
axes[1].legend(names_3_uses, loc='lower left')

#VH plots
s1_pd_ascending[VH_cols_3].plot(ax= axes[2], cmap = colors_3_uses)
axes[2].set_xlabel('')
axes[2].set_ylabel(r'$\sigma^0_{\rm VH,asc}$ [dB]')
axes[2].legend(names_3_uses, loc='lower left')
s1_pd_descending[VH_cols_3].plot(ax=axes[3], cmap = colors_3_uses)
axes[3].set_xlabel('')
axes[3].set_ylabel(r'$\sigma^0_{\rm VH,desc}$ [dB]')
axes[3].legend(names_3_uses, loc='lower left')
fig.supxlabel('Timestamp')
if write_fig:
    fig.savefig('Figures/Figures_chapter_SAR/s0_timeseries_vierluik.pdf', format = 'pdf')
fig

Make a Figure displaying the difference between the two 

In [ ]:
s1_diff_ascending_VV = s1_pd_g0_ascending[VV_cols_3] - s1_pd_ascending[VV_cols_3]
s1_diff_descending_VV = s1_pd_g0_descending[VV_cols_3] - s1_pd_descending[VV_cols_3]
s1_diff_ascending_VH = s1_pd_g0_ascending[VH_cols_3] - s1_pd_ascending[VH_cols_3]
s1_diff_descending_VH = s1_pd_g0_descending[VH_cols_3] - s1_pd_descending[VH_cols_3]
fig, axes = plt.subplots(4,1,figsize = (10,10),constrained_layout = True)
#VV plots
s1_diff_ascending_VV.plot(ax= axes[0], cmap = colors_3_uses)
axes[0].set_xlabel('')
axes[0].set_ylabel(r'$\gamma^0_{\rm VV,asc} - \sigma^0_{\rm VV,asc}$ [dB]')
axes[0].legend(names_3_uses, loc='lower left')
s1_diff_descending_VV.plot(ax=axes[1], cmap = colors_3_uses)
axes[1].set_xlabel('')
axes[1].set_ylabel(r'$\gamma^0_{\rm VV,desc} - \sigma^0_{\rm VV,desc}$ [dB]')
axes[1].legend(names_3_uses, loc='lower left')

#VH plots
s1_diff_ascending_VH.plot(ax= axes[2], cmap = colors_3_uses)
axes[2].set_xlabel('')
axes[2].set_ylabel(r'$\gamma^0_{\rm VH,asc} - \sigma^0_{\rm VH,asc}$ [dB]')
axes[2].legend(names_3_uses, loc='lower left')
s1_diff_descending_VH.plot(ax=axes[3], cmap = colors_3_uses)
axes[3].set_xlabel('')
axes[3].set_ylabel(r'$\gamma^0_{\rm VH,desc} - \sigma^0_{\rm VH,desc}$ [dB]')
axes[3].legend(names_3_uses, loc='lower left')
fig.supxlabel('Timestamp')
if write_fig:
    fig.savefig('Figures/Figures_chapter_SAR/diff_vierluik.pdf', format = 'pdf')
fig


In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# s1_diff_ascending_VV = scaler.fit_transform(s1_pd_g0_ascending[VV_cols_3]) - scaler.fit_transform(s1_pd_ascending[VV_cols_3])
# s1_diff_descending_VV = scaler.fit_transform(s1_pd_g0_descending[VV_cols_3]) - scaler.fit_transform(s1_pd_descending[VV_cols_3])
# s1_diff_ascending_VH = scaler.fit_transform(s1_pd_g0_ascending[VH_cols_3]) - scaler.fit_transform(s1_pd_ascending[VH_cols_3])
# s1_diff_descending_VH = scaler.fit_transform(s1_pd_g0_descending[VH_cols_3]) - scaler.fit_transform(s1_pd_descending[VH_cols_3])
# fig, axes = plt.subplots(4,1,figsize = (10,10),constrained_layout = True)
# #VV plots
# s1_diff_ascending_VV.plot(ax= axes[0], cmap = colors_3_uses)
# axes[0].set_xlabel('')
# axes[0].set_ylabel(r'$\sigma^0_{\rm VV,asc}$ [dB]')
# axes[0].legend(names_3_uses, loc='lower left')
# s1_diff_descending_VV.plot(ax=axes[1], cmap = colors_3_uses)
# axes[1].set_xlabel('')
# axes[1].set_ylabel(r'$\sigma^0_{\rm VV,desc}$ [dB]')
# axes[1].legend(names_3_uses, loc='lower left')

# #VH plots
# s1_diff_ascending_VH.plot(ax= axes[2], cmap = colors_3_uses)
# axes[2].set_xlabel('')
# axes[2].set_ylabel(r'$\sigma^0_{\rm VH,asc}$ [dB]')
# axes[2].legend(names_3_uses, loc='lower left')
# s1_diff_descending_VH.plot(ax=axes[3], cmap = colors_3_uses)
# axes[3].set_xlabel('')
# axes[3].set_ylabel(r'$\sigma^0_{\rm VH,desc}$ [dB]')
# axes[3].legend(names_3_uses, loc='lower left')
# fig.supxlabel('Timestamp')